# Instalacion de Dependencias



In [1]:
# !pip install pyspark
# !pip install -U -q PyDrive
# !apt update
# !apt install openjdk-8-jdk-headless -qq
# !apt install default-jre
# !apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Creacion del Spark Context

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Primer Cuatrimestre de 2018. Examen parcial, primera oportunidad.

Nintendo of America (EEUU) tiene
información de ventas de videojuegos
físicas mensuales totalizadas en EEUU
las cuales se realizan en cadenas de
tiendas de videojuegos en el siguiente
RDD: (id_videojuego, id_tienda, mes,
anio, total_ventas_mensuales).

Por otro lado tenemos un RDD con
información de las tiendas y de su
ubicación (id_tienda, direccion, latitud,
longitud, codigo_postal, estado).
Con esta información escribir un
programa en pySpark para obtener la
tienda que realizó menor cantidad de
ventas en el estado de "Georgia" en todo
el año 2017. 

Criterio: Es importante filtrar los datos que son necesarios antes de comenzar a trabajar, si no lo hacen se descuenta un min de 5ptos. Hay descuentos de 3
ptos si realizan operaciones de mas, o ineficientes (por ejemplo realizar un takeordered cuando necesitan solo obtener mínimo).
Si los formatos para realizar el join no se corresponde a (K, V) descuento de 5 puntos. 

In [3]:
sales = [
    (1, 1, '01', '2017', 500), # sera la minima
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 1, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (4, 3, '01', '2017', 500),
    (4, 3, '01', '2017', 500),
    (4, 3, '02', '2017', 500),
    (4, 3, '03', '2017', 500),    

]

stores = [
    (1 , 'address 1', -1, -1, '30002', 'Georgia'),
    (2 , 'address 2', -2, -2, '30003', 'Georgia'),
    (3 , 'address 2', -3, -3, '30004', 'Georgia'),
    (4 , 'address 2', -4, -4, '10119', 'New York')    
]

In [4]:
# Obtener la tienda que realizó menor cantidad de ventas en el estado de "Georgia" en todo el año 2017

In [5]:
salesRdd = sc.parallelize(sales)
storesRdd = sc.parallelize(stores)

In [6]:
salesToJoin = salesRdd.filter(lambda x: x[3] == '2017')\
      .map(lambda x: (x[1],x[4]))

In [7]:
storesToJoin = storesRdd.filter(lambda x: x[5] == 'Georgia')\
      .map(lambda x: (x[0], x[1]))

In [8]:
salesToJoin.take(20)

[(1, 500),
 (2, 500),
 (2, 500),
 (2, 500),
 (2, 500),
 (3, 500),
 (3, 500),
 (3, 500),
 (3, 500),
 (3, 500),
 (3, 500),
 (3, 500),
 (3, 500),
 (3, 500)]

In [9]:
storesToJoin.take(20)

[(1, 'address 1'), (2, 'address 2'), (3, 'address 2')]

In [11]:
salesToJoin.join(storesToJoin).take(4)

[(1, (500, 'address 1')),
 (2, (500, 'address 2')),
 (2, (500, 'address 2')),
 (2, (500, 'address 2'))]

In [10]:
salesToJoin.join(storesToJoin)\
  .reduceByKey(lambda a, b: (a[0]+b[0],a[1])).reduce(lambda x, y: x if x[1][0] < y[1][0] else y)

(1, (500, 'address 1'))

## Primer Cuatrimestre de 2018. Examen parcial, tercera oportunidad

El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta información se encuentra guardada en un archivo con el siguiente formato: (fecha, id_caso, descripción, estado_caso, categoría, latitud, longitud).

Los posibles estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolución. Las fechas se encuentran en el formato YYYY-MM-DD. 

Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un archivo con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él.
El sector encargado de las estadísticas oficiales del GCPD quiere analizar las siguientes situaciones:

a)  Las categorías que hayan incrementado su tasa de resolución en al menos un 10%  en el último trimestre, con respecto al trimestre anterior.
b) Tasa de participación de Batman por categoría, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto, y robo de vehículos) 

In [12]:
# (fecha, id_caso, descripción, estado_caso, categoría, latitud, longitud).
cases = [("2019-01-01", 1, "case 1", 2, "otro delito", -1, -1), 
         ("2019-06-01", 2, "case 2", 2, "robo", -1, -1),
         ("2019-06-01", 3, "case 2", 3, "robo", -1, -1),         
         ("2019-06-01", 4, "case 2", 1, "robo", -1, -1),         
         ("2019-06-01", 5, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 6, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 7, "case 2", 2, "robo", -1, -1),         
         ("2019-09-01", 8, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 9, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 10, "case 2", 3, "robo", -1, -1),
         ("2019-09-01", 60, "case 2", 3, "robo", -1, -1),
         ("2019-09-01", 70, "case 2", 3, "robo", -1, -1),         
         ("2019-09-01", 80, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 90, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 100, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 600, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 700, "case 2", 3, "robo", -1, -1),         
         ("2019-09-01", 800, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 900, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 1000, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 6000, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 7000, "case 2", 2, "robo", -1, -1),         
         ("2019-09-01", 8000, "case 2", 3, "robo", -1, -1),
         ("2019-09-01", 9000, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 10000, "case 2", 2, "robo", -1, -1),
         ("2019-06-01", 92, "case 2", 2, "hurto", -1, -1),
         ("2019-06-01", 93, "case 2", 3, "hurto", -1, -1),         
         ("2019-06-01", 94, "case 2", 3, "hurto", -1, -1),         
         ("2019-06-01", 95, "case 2", 3, "hurto", -1, -1),
         ("2019-09-01", 96, "case 2", 2, "hurto", -1, -1),
 
        ]

# (id_caso, respuesta)
batsignal = [(1,1),
         (2,1),
         (3,0),
         (4,0),
         (5,1),
         (6,0),
         (7,1),
         (8,0),
         (9,1),
         (10,0),         
         (60,0),
         (70,1),
         (80,1),
         (90,1),
         (100,1),
         (600,0),
         (700,1),
         (800,0),
         (900,1),
         (1000,1),
         (6000,0),
         (7000,1),
         (8000,0),
         (9000,1),
         (10000,1),
         (92,0),
         (93,0),             
         (94,0),
         (95,0),             
         (96,1)             
        ]

In [13]:
casesRdd = sc.parallelize(cases)
batsignalRdd = sc.parallelize(batsignal)

In [38]:
def to_trimestre(fecha):
    mes = int(fecha.split("-")[1])
    trimestre = 1
    if mes in range(4,7):
        trimestre = 2
    elif mes in range(7,10):
        trimestre = 3
    elif mes in range(10,13):
        trimestre = 4
    return trimestre

In [42]:
casesRdd.filter(lambda x: to_trimestre(x[0]) in range(2,4)).collect()

[('2019-06-01', 2, 'case 2', 2, 'robo', -1, -1),
 ('2019-06-01', 3, 'case 2', 3, 'robo', -1, -1),
 ('2019-06-01', 4, 'case 2', 1, 'robo', -1, -1),
 ('2019-06-01', 5, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 6, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 7, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 8, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 9, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 10, 'case 2', 3, 'robo', -1, -1),
 ('2019-09-01', 60, 'case 2', 3, 'robo', -1, -1),
 ('2019-09-01', 70, 'case 2', 3, 'robo', -1, -1),
 ('2019-09-01', 80, 'case 2', 1, 'robo', -1, -1),
 ('2019-09-01', 90, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 100, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 600, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 700, 'case 2', 3, 'robo', -1, -1),
 ('2019-09-01', 800, 'case 2', 1, 'robo', -1, -1),
 ('2019-09-01', 900, 'case 2', 1, 'robo', -1, -1),
 ('2019-09-01', 1000, 'case 2', 1, 'robo', -1, -1),
 ('2019-09-01', 6000, 'case 2', 2, 'robo', -1, -1),